# Hopfield Network for error detection in patterns

In [ ]:
from imageio import imread
import numpy as np
import matplotlib.pyplot as plt
import copy
from skimage import data, io, filters
from PIL import Image
from skimage.transform import resize
import random
%matplotlib inline

In [ ]:
def IsScalar(x):
    if type(x) in (list, np.ndarray,):
        return False
    else:
        return True

def get_route_indices(v):
    indices = []
    for j in range(v.shape[1]):
        indices.append(np.argmax(v[:, j]))
    indices.append(np.argmax(v[:, 0]))
    return indices
    
def total_distance(v):
    city_x_final = np.zeros((N+1))
    city_y_final = np.zeros((N+1))

    for j in range(N):
        for i in range(N):
            if v[i,j] == 1:
                city_x_final[j] = city_x[i]
                city_y_final[j] = city_y[i]

    city_x_final[N] = city_x_final[0]
    city_y_final[N] = city_y_final[0]

    td = 0
    
    for i in range(N-1):
        td += np.sqrt((city_x_final[i] - city_x_final[i+1])**2
                    + (city_y_final[i] - city_y_final[i+1])**2)
    
    td += np.sqrt((city_x_final[N-1] - city_x_final[0])**2
                + (city_y_final[N-1] - city_y_final[0])**2)
    
    return(td, city_x_final, city_y_final)

def Thresh(x):
    if IsScalar(x):
        val = 1 if x>0 else -1
    else:
        val = np.ones_like(x)
        val[x<0] = -1.
    return val

def Hamming(x, y):
    d = []
    for xx, yy in zip(x,y):
        dd = 0.
        for xxx,yyy in zip(xx,yy):
            if xxx==1 and yyy!=1:
                dd += 1.
            elif yyy==1 and xxx!=1:
                dd += 1.
        d.append(dd)
    return d

def Perturb(x, p=0.1):
    y = copy.deepcopy(x)
    for yy in y:
        for k in range(len(yy)):
            if np.random.rand()<p:
                yy[k] = Thresh(np.random.randint(2)*2-1)
    return y

def Update(W, x, b):
    xnew = x @ W - b
    return Thresh(xnew)

def get_route(v):
    route = ""
    for j in range(v.shape[1]):
        route += str(np.argmax(v[:, j])) + ' -> '
    return (route + str(np.argmax(v[:, 0])))

def hopfield():
    u0 = 0.02
    toend = 0
    udao = np.zeros((N, N))
    ctr = 0
    while toend == 0:
        ctr += 1
        v = np.random.rand(N,N)
        u = np.ones([N, N])*(-u0*np.log(N-1)/2)

        u += u*0.91 
        
        for _ in range(100000):
            for vx in range(N):
                for vi in range(N):
                    j1, j2, j3, j4 = 0, 0, 0, 0
                    
                    for j in range(N):
                        if j != vi:
                            j1 += v[vx, j]
                    j1 *= -A

                    for y in range(N):
                        if y != vx:
                            j2 += v[y, vi]
                    j2 *= -B

                    j3 = np.sum(v)
                    j3 = -C*(j3-N)

                    for y in range(N):
                        if y != vx:
                            if vi == 0:
                                j4 += d[vx, y]*(v[y, vi+1]+v[y, N-1])
                            elif vi == N-1:
                                j4 += d[vx, y]*(v[y, vi-1]+v[y, 0])
                            else:
                                j4 += d[vx, y]*(v[y, vi+1]+v[y, vi-1])
                    j4 *= -D
                    udao[vx, vi] = -u[vx, vi]+j1+j2+j3+j4

            u = u + alpha*udao
            v = (1+np.tanh(u/u0)) / 2
         
            for vx in range(N):
                for vi in range(N):
                    if(v[vx, vi] < 0.7):
                        v[vx, vi] = 0
                    if(v[vx, vi] >= 0.7):
                        v[vx, vi]=1
            
        t1, t2, t3 = 0, 0, 0
        
        for vx in range(N):
            for vi in range(N):
                t1+=v[vx, vi]

        t2=0
        
        for x in range(N):
            for i in range(N-1):
                for j in range(i+1, N):
                    t2+=np.multiply(v[x, i], v[x, j])

        t3=0
        
        for i in range(N):
            for x in range(N-1):
                for y in range(x+1, N):
                    t3+=np.multiply(v[x, i], v[y, i])

        if t1 == N and t2 == 0 and t3 == 0:
            toend = 1
        else:
            toend = 0

    return(v, ctr)

In [ ]:
letters = []
letters.append(imread('A.png'))
letters.append(imread('B.png'))
letters.append(imread('C.png'))
letters.append(imread('D.png'))

n = len(letters)
N = len(letters[0].flatten())
X = np.zeros((n, N))

for idx,img in enumerate(letters):
    X[idx,:] = Thresh(np.array([img.flatten()-0.5]))

In [ ]:
plt.figure(figsize=(16,4))

for k in range(n):
    plt.subplot(1,n,k+1);
    plt.imshow(np.reshape(X[k], (6,6)), cmap='tab20c_r'); plt.axis('off');

In [ ]:
b = np.zeros((1,N))
b = np.sum(X, axis=0) / n
W = ( X.T @ X ) / n - np.eye(N)
W0 = copy.deepcopy(W)

In [ ]:
k = np.random.randint(n)
Y = Perturb(X , p=0.2)
x = Y[k:k+1,]
x[0,24:] = -1.
err = Hamming(x, X[k:k+1,:])
print('Class '+str(k)+' with '+str(err)+' errors')
x_orig = copy.deepcopy(x)
plt.imshow(np.reshape(x,[6,6]), cmap='tab20c_r'); plt.axis('off');

In [ ]:
xs = copy.deepcopy(x_orig)
xa = copy.deepcopy(x)
n_iters = 2

In [ ]:
for idx in range(n_iters):
    xs = Update(W, xs, b)

In [ ]:
n_iters = 10

In [ ]:
for count in range(n_iters):
    node_idx = list(range(N))
    np.random.shuffle(node_idx)
    for idx in node_idx:
        ic = xa@W[:,idx] - b[idx]
        xa[0,idx] = Thresh(ic)

In [ ]:
print('Correct class is '+str(k))

In [ ]:
print('Synchronous updating')
for idx,t in enumerate(X):
    ds = Hamming(xs, [t])[0]
    print('Memory '+str(idx)+' has error '+str(ds))

In [ ]:
print('Asynchronous updating')
for idx,t in enumerate(X):
    da = Hamming(xa, [t])[0]
    print('Memory '+str(idx)+' has error '+str(da))

In [ ]:
plt.subplot(1,2,1); plt.imshow(np.reshape(xs,[6,6]), cmap='tab20c_r'); plt.title('Synchronous'); plt.axis('off')

In [ ]:
plt.subplot(1,2,2); plt.imshow(np.reshape(xa,[6,6]), cmap='tab20c_r'); plt.title('Asynchronous'); plt.axis('off')

# **Traveling** **Salesman** **Problem**


In [ ]:
N = 10

city_x = np.random.rand((10))
city_y = np.random.rand((10))

In [ ]:
print('The co-ordinates of the 10 cities are:')
for city in zip(city_x, city_y):
    print(city)

In [ ]:
plt.plot(city_x, city_y, 'o')
plt.title('Map of cities')

In [ ]:
d = np.zeros((N,N))

for i in range(N):
    for j in range(N):
        d[i, j] = np.sqrt((city_x[i] - city_x[j])**2 + (city_y[i] - city_y[j])**2)

In [ ]:
print(d)

In [ ]:
A = 500 
B = 500 
C = 1000
D = 500
alpha = 0.0001

In [ ]:
v = np.zeros([N,N])

ct = 0

min_dist = 20
best_path = None

for i in range(10):
    v, steps = hopfield()
    td, _, _ = total_distance(v)
    print(f"Epoch {i}: Ran for {steps} steps, total distance {td}")
    if td < min_dist:
        min_dist = td
        best_path = v

In [ ]:
print(min_dist)

In [ ]:
print(best_path)

In [ ]:
print(get_route(best_path))

In [ ]:
indices = get_route_indices(best_path)
print(indices)

In [ ]:
for i in indices[1:]:
    plt.plot([city_x[i], city_x[i-1]], [city_y[i], city_y[i-1]], '-')
plt.show()